In [1]:
# bùa chú
import sys
sys.path.append("/home/vanltt4/.local/lib/python3.8/site-packages")
sys.path

['/home/vanltt4/src/DeezyMatch-master',
 '/usr/local/anaconda3/lib/python38.zip',
 '/usr/local/anaconda3/lib/python3.8',
 '/usr/local/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/usr/local/anaconda3/lib/python3.8/site-packages',
 '/usr/local/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/home/vanltt4/.ipython',
 '/home/vanltt4/.local/lib/python3.8/site-packages']

# DeezyMatch, example_001

## Model inference

In [ ]:
from DeezyMatch import inference as dm_inference

# model inference using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./models/finetuned_trans_09052022/input_dfm_rnn.yaml",
             dataset_path="./dataset/dataset-string-matching_test.txt", 
             pretrained_model_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.model", 
             pretrained_vocab_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.vocab")

## Generate query vectors

In [7]:
from DeezyMatch import inference as dm_inference

# generate vectors for queries (specified in dataset_path) 
# using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./models/finetuned_trans_09052022/input_dfm_rnn.yaml",
            dataset_path="./dataset/dataset-ULAN-queries.txt", 
            pretrained_model_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.model", 
            pretrained_vocab_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.vocab",
            inference_mode="vect",
            scenario="queries/ULAN")

2022-05-14 04:47:57 u116614 [INFO] read input file: ./models/finetuned_trans_09052022/input_dfm_rnn.yaml
2022-05-14 04:47:57 u116614 [INFO] pytorch will use: cuda
2022-05-14 04:47:59 u116614 [INFO] read CSV file: ./dataset/dataset-ULAN-queries.txt
2022-05-14 04:47:59 u116614 [INFO] number of labels, True: 29 and False: 0
2022-05-14 04:47:59 u116614 [INFO] skipping 0 lines


2022-05-14 04:47:59 u116614 [INFO] save test-data-class: /home/vanltt4/src/DeezyMatch-master/queries/ULAN/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]

--- 2.1560702323913574 seconds ---


## Generate candidate vectors

In [12]:
from DeezyMatch import inference as dm_inference

# generate vectors for candidates (specified in dataset_path) 
# using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./models/finetuned_trans_09052022/input_dfm_rnn.yaml",
             dataset_path="./dataset/dataset-ULAN-candidates.txt", 
             pretrained_model_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.model", 
             pretrained_vocab_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.vocab",
             inference_mode="vect",
             scenario="candidates/ULAN")

2022-05-14 04:49:15 u116614 [INFO] read input file: ./models/finetuned_trans_09052022/input_dfm_rnn.yaml
2022-05-14 04:49:15 u116614 [INFO] pytorch will use: cuda
2022-05-14 04:49:17 u116614 [INFO] read CSV file: ./dataset/dataset-ULAN-candidates.txt
2022-05-14 04:49:17 u116614 [INFO] number of labels, True: 73 and False: 0
2022-05-14 04:49:17 u116614 [INFO] skipping 0 lines


2022-05-14 04:49:17 u116614 [INFO] save test-data-class: /home/vanltt4/src/DeezyMatch-master/candidates/ULAN/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]

--- 2.2505648136138916 seconds ---


## Transformer- Assembling candidates/queries vector representations

In [13]:
from DeezyMatch import combine_vecs_trans

# combine vectors stored in candidates/test and save them in combined/candidates_test
combine_vecs_trans(model_name="finetuned_trans_09052022", 
             input_scenario='candidates/ULAN', 
             output_scenario='combined/candidates_ULAN', 
             print_every=10)

2022-05-14 04:49:21 u116614 [INFO] read input file: candidates/ULAN/input_dfm_rnn.yaml
2022-05-14 04:49:21 u116614 [INFO] pytorch will use: cuda


-- Combine vectors
Reading vectors from candidates/ULAN/embeddings/finetuned_trans_09052022_vecs_*
list_files:  ['candidates/ULAN/embeddings/finetuned_trans_09052022_vecs_0']
0000000 candidates/ULAN/embeddings/finetuned_trans_09052022_vecs_0
0


-- Combine IDs
0000000 candidates/ULAN/embeddings/finetuned_trans_09052022_indxs_0

--- 169.3242790699005 seconds ---


In [14]:
from DeezyMatch import combine_vecs_trans

# combine vectors stored in candidates/test and save them in combined/candidates_test
combine_vecs_trans(model_name="finetuned_trans_09052022", 
             input_scenario='queries/ULAN', 
             output_scenario='combined/queries_ULAN', 
             print_every=10)

2022-05-14 04:49:26 u116614 [INFO] read input file: queries/ULAN/input_dfm_rnn.yaml
2022-05-14 04:49:26 u116614 [INFO] pytorch will use: cuda


-- Combine vectors
Reading vectors from queries/ULAN/embeddings/finetuned_trans_09052022_vecs_*
list_files:  ['queries/ULAN/embeddings/finetuned_trans_09052022_vecs_0']
0000000 queries/ULAN/embeddings/finetuned_trans_09052022_vecs_0
0


-- Combine IDs
0000000 queries/ULAN/embeddings/finetuned_trans_09052022_indxs_0

--- 173.81107902526855 seconds ---


## Candidate Ranker

In [19]:
from DeezyMatch import candidate_ranker_trans

# Select candidates based on L2-norm distance (aka faiss distance):
# find candidates from candidate_scenario 
# for queries specified in query_scenario
candidates_pd = \
    candidate_ranker_trans(model_name="finetuned_trans_09052022",
                     query_scenario="./combined/queries_ULAN",
                     candidate_scenario="./combined/candidates_ULAN", 
                     ranking_metric="cosine", 
                     selection_threshold=0.5, 
                     num_candidates=2, 
                     search_size=5, 
                     output_path="ranker_results/ULAN_candidates_deezymatch", 
                     pretrained_model_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.model", 
                     pretrained_vocab_path="./models/finetuned_trans_09052022/finetuned_trans_09052022.vocab", 
                     number_test_rows=20)

2022-05-14 04:53:43 u116614 [INFO] read input file: ./combined/candidates_ULAN/input_dfm_rnn.yaml
2022-05-14 04:53:43 u116614 [INFO] pytorch will use: cuda
Is faiss_id_candis already trained? True
=========== Start the search for 0 Van Le Tran Truc
all_candidates:  ['le tran truc van' 'macclure victor' 'laure tixier' 'max truninger'
 'adam callander']


  0%|          | 0/1 [00:00<?, ?it/s]

all_preds: tensor([0.1202, 0.2691, 0.1632, 0.3000, 0.3039], device='cuda:0')
ID: 1/29 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 1 Van Le
all_candidates:  ['le roy' 'moore hal' 'robit' 'hà hồ' 'iutz']


  0%|          | 0/1 [00:00<?, ?it/s]

all_preds: tensor([0.1352, 0.9655, 0.2485, 0.9736, 0.8865], device='cuda:0')
ID: 2/29 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 2 Hồ Ngọc Hà
all_candidates:  ['hà hồ' 'kim chanjoong' 'chen qingqing' 'park jonathan'
 'edwin calligan']


...

all_preds: tensor([0.6969, 0.6624, 0.3377, 0.5314, 0.6070], device='cuda:0')
ID: 18/29 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 18 היפוקראטס
all_candidates:  ["אלורה ג'ניפר" 'hà hồ' 'ippocrate' 'le roy' 'robit']


  0%|          | 0/1 [00:00<?, ?it/s]

all_preds: tensor([0.3083, 0.9587, 0.4616, 0.0318, 0.5161], device='cuda:0')
ID: 19/29 -- Number of found candidates so far: 1, searched: 5
=========== Start the search for 19 Crocchia Girolamo
all_candidates:  ['macconnel kim r.' 'macconnel kim' 'emanoel araújo' 'il crocicchia'
 'adam callander']


  0%|          | 0/1 [00:00<?, ?it/s]

all_preds: tensor([0.2054, 0.1423, 0.1140, 0.1811, 0.0518], device='cuda:0')
ID: 20/29 -- Number of found candidates so far: 4, searched: 5
TOTAL TIME: 8.305619239807129


In [20]:
candidates_pd

,query,pred_score,1-pred_score,faiss_distance,cosine_dist,candidate_original_ids,query_original_id,num_all_searches
id,,,,,,,,
0,Van Le Tran Truc,"{'Le Tran Truc Van': 0.1202, 'Macclure Victor'...","{'Le Tran Truc Van': 0.8798, 'Macclure Victor'...","{'Le Tran Truc Van': 6.4978, 'Macclure Victor'...","{'Le Tran Truc Van': 0.1866, 'Macclure Victor'...","{'Le Tran Truc Van': 2, 'Macclure Victor': 33}",0,5
1,Van Le,"{'Le Roy': 0.1352, 'Moore Hal': 0.9655}","{'Le Roy': 0.8648, 'Moore Hal': 0.034499999999...","{'Le Roy': 8.1601, 'Moore Hal': 9.4413}","{'Le Roy': 0.3733, 'Moore Hal': 0.4197}","{'Le Roy': 37, 'Moore Hal': 28}",1,5
2,Hồ Ngọc Hà,"{'Kim ChanJoong': 0.128, 'Hà Hồ': 0.3796}","{'Kim ChanJoong': 0.872, 'Hà Hồ': 0.6204000000...","{'Kim ChanJoong': 9.6501, 'Hà Hồ': 9.2395}","{'Kim ChanJoong': 0.4294, 'Hà Hồ': 0.4784}","{'Kim ChanJoong': 56, 'Hà Hồ': 0}",2,5
3,Ho Ngoc Ha,{'Hà Hồ': 0.0839},{'Hà Hồ': 0.9161},{'Hà Hồ': 6.067},{'Hà Hồ': 0.307},{'Hà Hồ': 0},3,5
4,Đàm Vĩnh Hưng,"{'Kim ChanJoong': 0.4382, 'Thomas Sutton': 0.3...","{'Kim ChanJoong': 0.5618000000000001, 'Thomas ...","{'Kim ChanJoong': 10.0206, 'Thomas Sutton': 11...","{'Kim ChanJoong': 0.3922, 'Thomas Sutton': 0.4...","{'Kim ChanJoong': 56, 'Thomas Sutton': 40}",4,5
5,jutz karl,"{'Jutz Carl': 0.3057, 'Iutz': 0.4372}","{'Jutz Carl': 0.6942999999999999, 'Iutz': 0.5628}","{'Jutz Carl': 3.9343, 'Iutz': 11.2349}","{'Jutz Carl': 0.1603, 'Iutz': 0.4507}","{'Jutz Carl': 3, 'Iutz': 4}",5,5
6,John Howell-Books,"{'unknown Welsh': 0.529, 'John Howell': 0.4841}","{'unknown Welsh': 0.471, 'John Howell': 0.5159}","{'unknown Welsh': 8.5938, 'John Howell': 8.969}","{'unknown Welsh': 0.259, 'John Howell': 0.2888}","{'unknown Welsh': 67, 'John Howell': 5}",6,5
7,Davidson Bruce Landon,"{'Guttuso Aldo Renato': 0.4536, 'Adams Francis...","{'Guttuso Aldo Renato': 0.5464, 'Adams Francis...","{'Guttuso Aldo Renato': 11.9695, 'Adams Franci...","{'Guttuso Aldo Renato': 0.3241, 'Adams Francis...","{'Guttuso Aldo Renato': 42, 'Adams Francis Edw...",7,5
8,Allora Jennifer,"{'Emanoel Araújo': 0.7378, 'Emanuel Araujo': 0...","{'Emanoel Araújo': 0.2622, 'Emanuel Araujo': 0...","{'Emanoel Araújo': 10.5977, 'Emanuel Araujo': ...","{'Emanoel Araújo': 0.3168, 'Emanuel Araujo': 0...","{'Emanoel Araújo': 26, 'Emanuel Araujo': 25}",8,5


## Candidate ranking on-the-fly

In [ ]:
from DeezyMatch import candidate_ranker

# Ranking on-the-fly
# find candidates from candidate_scenario 
# for queries specified by the `query` argument
candidates_pd = \
    candidate_ranker(candidate_scenario="./combined/candidates_test",
                     query=["DeezyMatch", "kasra", "fede", "mariona"],
                     ranking_metric="faiss", 
                     selection_threshold=5., 
                     num_candidates=1, 
                     search_size=100, 
                     output_path="ranker_results/test_candidates_deezymatch_on_the_fly", 
                     pretrained_model_path="./models/finetuned_test001/finetuned_test001.model", 
                     pretrained_vocab_path="./models/finetuned_test001/finetuned_test001.vocab", 
                     number_test_rows=20)

In [ ]:
candidates_pd

### The candidate ranker can be initialised, to be used multiple times, by running:

In [ ]:
from DeezyMatch import candidate_ranker_init

# initializing candidate_ranker via candidate_ranker_init
myranker = candidate_ranker_init(candidate_scenario="./combined/candidates_test",
                                 query=["DeezyMatch", "kasra", "fede", "mariona"],
                                 ranking_metric="faiss", 
                                 selection_threshold=5., 
                                 num_candidates=1, 
                                 search_size=100, 
                                 output_path="ranker_results/test_candidates_deezymatch_on_the_fly", 
                                 pretrained_model_path="./models/finetuned_test001/finetuned_test001.model", 
                                 pretrained_vocab_path="./models/finetuned_test001/finetuned_test001.vocab", 
                                 number_test_rows=20)

In [ ]:
# print the content of myranker by:
print(myranker)

In [ ]:
# To rank the queries:
myranker.rank()

In [ ]:
#The results are stored in:
myranker.output

### Change the queries

In [ ]:
myranker.set_query(query=["khan", "feng", "cheng"])

In [ ]:
# To rank the queries:
myranker.rank()

In [ ]:
#The results are stored in:
myranker.output

In [6]:
candidate = set()
with open("./dataset/finetuned-test.txt","r", encoding="utf-8") as fin:
    i = 0
    while(i < 1000):
        line = fin.readline()
        if not line:
            break
        token = line.split("\t")
        candidate.add(token[1])
        i = i + 1

with open("./dataset/dataset-ULAN1-candidates.txt", "w", encoding="utf-8") as fout:
    for cand in candidate:
        fout.write(cand)
        fout.write("\n")